In [0]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.1.0


In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/cleansed_data.csv"
DATA_FILE_NAME = 'My Drive/Colab Notebooks/input.xlsx'
CLEANSED_FILE_DIR = '/My Drive/Colab Notebooks/'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [0]:
import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Activation, Bidirectional, Flatten, BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [0]:
def load_doc(filename):
  if(filename.endswith('.xlsx')):
    data_df = pd.read_excel(filename, lines=True)
  elif(filename.endswith('.csv')):
    data_df = pd.read_csv(filename, keep_default_na = False)
  return data_df

In [0]:
def encode_target(target):
  le = LabelEncoder()
  return le.fit_transform(target), le

def decode_prediction(pred, encoder):
  return encoder.inverse_transform(pred)

In [45]:
cleansed_data_df = load_doc(PROJECT_DIR)
print('shape of Data : ', cleansed_data_df.shape)

shape of Data :  (8500, 4)


In [46]:
cleansed_data_df.describe()

,short_description,description,combined_desc,assignment_group
count,8500,8500,8500,8500
unique,6000,5484,5634,74
top,job job failed job scheduler,,,GRP_0
freq,473,1123,1047,3976


In [47]:
cleansed_data_df.head()

,short_description,description,combined_desc,assignment_group
0,login issue,verified user details employee manager checked...,login issue verified user details employee man...,GRP_0
1,outlook,team meetings skype meetings etc not appearing...,outlook team meetings skype meetings etc not a...,GRP_0
2,cant log vpn,hi cannot log vpn,cant log vpn hi cannot log vpn,GRP_0
3,unable access hr tool page,unable access hr tool page,unable access hr tool page unable access hr to...,GRP_0
4,skype error,skype error,skype error skype error,GRP_0


In [0]:
labelencoder = LabelEncoder()
cleansed_data_df['assignment_group'] = labelencoder.fit_transform(cleansed_data_df['assignment_group'])

In [49]:
cleansed_data_df

,short_description,description,combined_desc,assignment_group
0,login issue,verified user details employee manager checked...,login issue verified user details employee man...,0
1,outlook,team meetings skype meetings etc not appearing...,outlook team meetings skype meetings etc not a...,0
2,cant log vpn,hi cannot log vpn,cant log vpn hi cannot log vpn,0
3,unable access hr tool page,unable access hr tool page,unable access hr tool page unable access hr to...,0
4,skype error,skype error,skype error skype error,0
...,...,...,...,...
8495,emails not coming zz mail,good afternoon not receiving emails sent zz ma...,emails not coming zz mail good afternoon not r...,22
8496,telephony software issue,telephony software issue,telephony software issue telephony software issue,0
8497,windows password reset tifpdchb pedxruyf,windows password reset tifpdchb pedxruyf,windows password reset tifpdchb pedxruyf windo...,0
8498,machine funcionando,unable access machine utilities finish drawers...,machine funcionando unable access machine util...,59


In [0]:
cleansed_data = cleansed_data_df

In [51]:
oneHotencoder = OneHotEncoder()

#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X = oneHotencoder.fit_transform(cleansed_data['assignment_group'].values.reshape(-1,1)).toarray()
print(X)
#print(cleansed_data.shape[1])
#To add this back into the original dataframe 
#dfOneHot = pd.DataFrame(X, columns = ["assignment_group_"+str(int(i)) for i in range(cleansed_data.shape[1])]) 
#df = pd.concat([cleansed_data, dfOneHot], axis=1)
#droping the country column 
#df= df.drop(['assignment_group'], axis=1) 
#printing to verify 
#print(df.head())


[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(cleansed_data_df['combined_desc'], X, test_size=0.2, shuffle= True)

In [77]:
print(y_train)


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [78]:
print(x_train[22])

unable connect vpn unable connect vpn


In [79]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical


y_train_cate = to_categorical(y_train)
y_test_cate = to_categorical(y_valid)
print(y_train_cate.shape)

x_train = x_train.astype(str)
x_valid = x_valid.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(x_train)

#converting text into integer sequences
x_tr_seq  = tokenizer.texts_to_sequences(x_train) 
x_val_seq = tokenizer.texts_to_sequences(x_valid)

#padding to prepare sequences of same length
x_tr_seq  = pad_sequences(x_tr_seq, maxlen=200)
x_val_seq = pad_sequences(x_val_seq, maxlen=200)


(6800, 74, 2)


In [81]:
print(x_train[24])
print(x_tr_seq[24])
#print(x_train[24])
#print(x_tr_seq[22])
#print(x_tr_seq[24])
#print(x_tr_seq[25])

vpn not working im not able connect company network vpn pls check cc sir not able upload result no company network
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0   

In [82]:
y_train = np.asarray(y_train)
print(y_train.shape)
type(y_train)

(6800, 74)


numpy.ndarray

**Pretrained Glove embeddings with 100d**

In [83]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('../content/drive/My Drive/Colab Notebooks/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [84]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

8730


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((size_of_vocabulary, 100))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import *

In [87]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=200,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 200, 100)          873000    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200, 256)          234496    
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 74)                9546      
Total params: 1,149,938
Trainable params: 1,149,938
Non-trainable params: 0
_________________________________________________________________
None


In [88]:
history = model.fit(x_tr_seq,y_train,batch_size=32,epochs=10,validation_data=(x_val_seq,y_valid),verbose=1,callbacks=[es,mc])

Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 123s 18ms/step - loss: 2.2670 - acc: 0.5249 - val_loss: 1.8869 - val_acc: 0.5576

Epoch 00001: val_acc improved from -inf to 0.55765, saving model to best_model.h5
Epoch 2/10
6800/6800 [==============================] - 118s 17ms/step - loss: 1.7299 - acc: 0.5829 - val_loss: 1.6837 - val_acc: 0.6041

Epoch 00002: val_acc improved from 0.55765 to 0.60412, saving model to best_model.h5
Epoch 3/10
6800/6800 [==============================] - 118s 17ms/step - loss: 1.5121 - acc: 0.6247 - val_loss: 1.5758 - val_acc: 0.6118

Epoch 00003: val_acc improved from 0.60412 to 0.61176, saving model to best_model.h5
Epoch 4/10
6800/6800 [==============================] - 118s 17ms/step - loss: 1.3534 - acc: 0.6509 - val_loss: 1.4970 - val_acc: 0.6224

Epoch 00004: val_acc improved from 0.61176 to 0.62235, saving model to best_model.h5
Epoch 5/10
6800/6800 [==============================] - 118s 17

In [89]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq,y_valid, batch_size=32)
print(val_acc)

1700/1700 [==============================] - 5s 3ms/step
0.6435294151306152


In [0]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train_vec, x_valid_vec, y_train_vec, y_valid_vec = train_test_split(cleansed_data_df['combined_desc'], X, test_size=0.2, shuffle= True)

In [97]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_vec)
y_test_cate = to_categorical(y_train_vec)
print(y_train_cate.shape)

x_train_vec = x_train_vec.astype(str)
x_valid_vec = x_valid_vec.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)
#preparing vocabulary
tokenizer.fit_on_texts(x_train_vec)

#converting text into integer sequences
x_tr_seq_vec  = tokenizer.texts_to_sequences(x_train_vec) 
x_val_seq_vec = tokenizer.texts_to_sequences(x_valid_vec)

#padding to prepare sequences of same length
x_tr_seq_vec  = pad_sequences(x_tr_seq_vec, maxlen=200)
x_val_seq_vec = pad_sequences(x_val_seq_vec, maxlen=200)



(6800, 74, 2)


In [98]:
size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

8699


In [0]:
from gensim.models.keyedvectors import KeyedVectors
PROJECT_DIR = "/content/drive/My Drive/Colab Notebooks/"
EMBEDDING_FILE = 'SO_vectors_200.bin'
EmbeddingFile = PROJECT_DIR+EMBEDDING_FILE
word2vec = KeyedVectors.load_word2vec_format(EmbeddingFile, binary=True)
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000


In [0]:
# create a weight matrix for words in training docs
EMBEDDING_DIM = 200
embedding_matrix_word2vec = np.zeros((size_of_vocabulary, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    if word in word2vec.vocab:
        embedding_matrix_word2vec[i] = word2vec.word_vec(word)   

In [101]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,200,weights=[embedding_matrix_word2vec],input_length=200,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 200)          1739800   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 200, 256)          336896    
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_16 (Dense)             (None, 74)                9546      
Total params: 2,119,138
Trainable params: 2,119,138
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
history = model.fit(x_tr_seq_vec,y_train_vec,batch_size=32,epochs=10,validation_data=(x_val_seq_vec,y_valid_vec),verbose=1,callbacks=[es,mc])

Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 179s 26ms/step - loss: 2.0983 - acc: 0.5482 - val_loss: 1.6734 - val_acc: 0.5988

Epoch 00001: val_acc improved from -inf to 0.59882, saving model to best_model.h5
Epoch 2/10
6800/6800 [==============================] - 175s 26ms/step - loss: 1.5635 - acc: 0.6162 - val_loss: 1.5037 - val_acc: 0.6247

Epoch 00002: val_acc improved from 0.59882 to 0.62471, saving model to best_model.h5
Epoch 3/10
6800/6800 [==============================] - 175s 26ms/step - loss: 1.3636 - acc: 0.6499 - val_loss: 1.4390 - val_acc: 0.6329

Epoch 00003: val_acc improved from 0.62471 to 0.63294, saving model to best_model.h5
Epoch 4/10
6800/6800 [==============================] - 175s 26ms/step - loss: 1.2025 - acc: 0.6815 - val_loss: 1.3834 - val_acc: 0.6447

Epoch 00004: val_acc improved from 0.63294 to 0.64471, saving model to best_model.h5
Epoch 5/10
6800/6800 [==============================] - 175s 26

In [103]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_vec,y_valid_vec, batch_size=32)
print(val_acc)

1700/1700 [==============================] - 10s 6ms/step
0.6617646813392639


**FASTTEXT pre trained embedding**

In [0]:
from sklearn.model_selection import train_test_split

# Split the data
x_train_fast, x_valid_fast, y_train_fast, y_valid_fast = train_test_split(cleansed_data_df['combined_desc'], X, test_size=0.2, shuffle= True)


In [105]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

y_train_cate = to_categorical(y_train_fast)
y_test_cate = to_categorical(y_valid_fast)
print(y_train_cate.shape)

x_train_fast = x_train_fast.astype(str)
x_valid_fast = x_valid_fast.astype(str)

#Tokenize the sentences
tokenizer = Tokenizer(num_words=10000,char_level=False)

#preparing vocabulary
tokenizer.fit_on_texts(list(x_train_fast))

#converting text into integer sequences
x_tr_seq_fast  = tokenizer.texts_to_sequences(x_train_fast) 
x_val_seq_fast = tokenizer.texts_to_sequences(x_valid_fast)

#padding to prepare sequences of same length
x_tr_seq_fast  = pad_sequences(x_tr_seq_fast, maxlen=300)
x_val_seq_fast = pad_sequences(x_val_seq_fast, maxlen=300)

size_of_vocabulary=len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)


(6800, 74, 2)
8703


In [0]:
max_features = 200000 #17780 #using all unique words
embedding_dim = 300
num_classes = 74
batch_size = 32

embedding_path = "/content/drive/My Drive/Colab Notebooks/wiki-news-300d-1M.vec"

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

word_index = tokenizer.word_index
nb_words = len(word_index)
embedding_matrix_fast = np.zeros((nb_words + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix_fast[i] = embedding_vector

In [110]:
model=Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,weights=[embedding_matrix_fast],input_length=300,trainable=True)) 

#lstm layer
model.add(Bidirectional(LSTM(128,return_sequences=True,dropout=0.2)))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(128,activation='relu'))  
model.add(Dense(74,activation='softmax'))  

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["acc"]) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 300, 300)          2610900   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 300, 256)          439296    
_________________________________________________________________
global_max_pooling1d_10 (Glo (None, 256)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_20 (Dense)             (None, 74)                9546      
Total params: 3,092,638
Trainable params: 3,092,638
Non-trainable params: 0
_________________________________________________________________
None


In [111]:
model.fit(x_tr_seq_fast,y_train_fast,batch_size=32,epochs=10,validation_data=(x_val_seq_fast,y_valid_fast),verbose=1,callbacks=[es,mc])

Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 319s 47ms/step - loss: 2.2178 - acc: 0.5369 - val_loss: 1.8581 - val_acc: 0.5782

Epoch 00001: val_acc improved from -inf to 0.57824, saving model to best_model.h5
Epoch 2/10
6800/6800 [==============================] - 322s 47ms/step - loss: 1.6067 - acc: 0.6097 - val_loss: 1.6667 - val_acc: 0.5953

Epoch 00002: val_acc improved from 0.57824 to 0.59529, saving model to best_model.h5
Epoch 3/10
6800/6800 [==============================] - 322s 47ms/step - loss: 1.3123 - acc: 0.6609 - val_loss: 1.5353 - val_acc: 0.6200

Epoch 00003: val_acc improved from 0.59529 to 0.62000, saving model to best_model.h5
Epoch 4/10
6800/6800 [==============================] - 322s 47ms/step - loss: 1.0729 - acc: 0.7193 - val_loss: 1.5609 - val_acc: 0.6106

Epoch 00004: val_acc did not improve from 0.62000
Epoch 5/10
6800/6800 [==============================] - 324s 48ms/step - loss: 0.8841 - acc: 0.770

In [112]:
#loading best model
from keras.models import load_model
model = load_model('best_model.h5')

#evaluation 
_,val_acc = model.evaluate(x_val_seq_fast,y_valid_fast, batch_size=32)
print(val_acc)

1700/1700 [==============================] - 16s 10ms/step
0.6370587944984436
